In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as plt
import statsmodels.api as sm
import pandas as pd
import csv
import math
import plotly.graph_objects as go


from pandas import DataFrame, Series
from sklearn.preprocessing import MinMaxScaler
from  scipy.stats import norm
from scipy.signal import find_peaks

%matplotlib inline

from scipy.signal import savgol_filter

In [2]:
def normalization(columns): # -1~1사이의 값으로 정규화
    MinMax = (columns - columns.min(axis=0)) / (columns.max(axis=0) - columns.min(axis=0))
    MinMax = (MinMax * 2) 
    MinMax
    return MinMax

In [3]:
def SVM(x, y, z):
    svm = []
    for i in range(0, len(x)):
        svm.append(math.sqrt(x[i]*x[i]+y[i]*y[i]+z[i]*z[i]))
    return svm

In [4]:
def LPF(rawData):
    rawData['targetData'] = rawData['AccZ']
#     result = []
    ts = 0.01 # 100Hz
    tau = 0.1
#     print_graph(rawData['targetData'], 50)
#     print(rawData['targetData'])
    tmpResult = Series(np.zeros(len(rawData.targetData)))
    tmpResult[0] = rawData.targetData[0]
#     tmpResult = Series(np.zeros(len(svm)))
#     tmpResult[0] = svm[0]

    for i in np.arange(1, len(rawData.targetData)):
        tmpResult[i] = (tau*tmpResult[i-1] + ts * rawData.targetData[i]) / (ts + tau)

    rawData['LPF'] = tmpResult

#     plt.plot(rawData.SampleRate, rawData.targetData, '#afafaf',  rawData.SampleRate, rawData.LPF_X, 'b')
#     plt.grid(True)
#     plt.show()
#     print(rawData)
    return rawData

In [5]:
def print_graph(MinMax, num): # num is cut range
    length = len(MinMax)
    plt.figure(figsize=(20, 5)) # graph size
    plt.xticks(np.arange(0, length+1, num))
    plt.plot(MinMax)
    plt.show()

In [6]:
def cut_count(MinMax, cnt): # 왼발, 오른발의 데이터를 잘라서 구분
    count = int((len(MinMax)) / cnt)
    step = []
    temp = 0
    for i in range(0, count):
        step.append(MinMax[temp:temp+cnt])
        temp = temp + cnt
    return step

In [7]:
def ACF(step, cnt): #자기상관계수 구하는 함수 cnt = 한발에 들어가는 데이터의 수
    answer = 0
    acf = []
    for i in step:
        answer = 0
#         answer = np.linalg.norm(i)
        for j in i:
#             print(j)
            answer = answer + j*j
        answer = answer / cnt
#         print(answer)
        acf.append(answer)
    return acf

In [8]:
def compare_acf(acf): # 자기상관계수 값을 구해줌
    num = len(acf) / 2
    left = []
    right = []
    result = []
#     print("acf_len : ", len(acf))
    avg = 0
    if (len(acf) == 2 or len(acf) == 3) :
        if (acf[0] > acf[1]):
            result = [[1.0, acf[1]/acf[0]],
                     [acf[1]/acf[0], 1.0]]
            return result
        else :
            result = [[1.0, acf[0]/acf[1]],
                     [acf[0]/acf[1], 1.0]]
            return result
    elif (len(acf) == 4 or len(acf) == 5) : 
        for i in range(1, 3):
            if (acf[i-1] > acf[i]):
                avg += acf[i]/acf[i-1]
                         
            else :
                avg += acf[i-1]/acf[i]
        result = [[1.0, avg / 2],
                  [avg/2, 1.0]]
        return result 
    if (len(acf) % 2 == 0) :
#         print("even")
        for i in range(1, len(acf), +2):
            left.append(acf[i])
            right.append(acf[i-1])
#         print(left, right)
        result = np.corrcoef(left, right)
        result = abs(result)
#         print("even", result)
    else :
#         print("odd")
        for i in range(1, len(acf)-1, +2):
            left.append(acf[i])
            right.append(acf[i-1])
#         print(left, right)
        result = np.corrcoef(left, right)
        result = abs(result)
#         print ("odd", result)
    return result

In [28]:
def main(rawData):
#     x_data = rawData['AccX']
#     y_data = rawData['AccY']
#     z_data = rawData['AccZ']
#     svm = SVM(x_data, y_data, z_data)
    lpf = LPF(rawData)
    yhat = normalization(savgol_filter(lpf.LPF, 51, 3)) # window size 51, polynomial order 3
#     print_graph(lpf.LPF, 50)
#     print_graph(yhat, 50)


    indices = find_peaks(yhat)[0]
    temp = []
    print('yhat : ', yhat)
    for i in indices :
        if (yhat[i] > 0.95):
            temp.append(i)
    peak = [temp[0]]
    print("Temp : ", temp)
    for i in range(1, len(temp)):
        if (temp[i]-temp[i-1] > 35):
#             if (yhat[temp[i]]>yhat[temp[i-1]]) :
            peak.append(temp[i])
#             else : peak.append(temp[i-1])
    print("peak : ", peak)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=yhat,
        mode='lines+markers',
        name='Original Plot'
    ))

    fig.add_trace(go.Scatter(
        x=peak,
        y=[yhat[j] for j in peak],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='cross'
        ),
        name='Detected Peaks'
    ))

    fig.show()
    distance = []
    value = 0
    for i in range(1, len(peak)):
        if (peak[i]-peak[i-1] > 40):
            distance.append(peak[i]-peak[i-1])
            value += (peak[i]-peak[i-1])
    print(len(distance))
    average = round(value / len(distance))
    avg = int(average/2)+1 
#     print_graph(yhat, avg)
    print("distance : ", distance, ", avg : ", average, type(average))

    step = cut_count(yhat,avg)
#     print("step_len : ", len(step))
    acf = ACF(step, avg)
#     print(acf)
    compare = compare_acf(acf)
#     print(compare)

    result = True
    if (compare[0][1] > 0.6):
        result=  True
    else : result = False
    if (result == True):
        print('\033[34m', result, '\033[0m', compare[0][1])
    else : print('\033[31m', result, '\033[0m', compare[0][1])

In [29]:
for i in range(1, 6):
    rawData = pd.read_csv(f'0529Data/0529_user1/AbnormalGait{i}.csv')
    main(rawData)

yhat :  [1.60331311 1.52260779 1.44145982 1.36012647 1.27886499 1.19793267
 1.11758675 1.03808451 0.9596832  0.8826401  0.80721246 0.73365755
 0.66223264 0.59319498 0.52680184 0.46331048 0.40297817 0.34606218
 0.29281976 0.24350818 0.19838471 0.1577066  0.12173113 0.09071555
 0.06491713 0.04459314 0.03413756 0.02936658 0.03191434 0.04426621
 0.068209   0.10604115 0.15897637 0.22486984 0.29989341 0.38041746
 0.46364956 0.54736564 0.63010163 0.71064115 0.78772724 0.85973352
 0.92476049 0.98103491 1.03061447 1.07703249 1.12159905 1.16310253
 1.19960178 1.22887228 1.24874202 1.25704756 1.25229651 1.2375362
 1.21773106 1.19755081 1.18184029 1.17209374 1.16782812 1.16706926
 1.16697899 1.16473772 1.1589867  1.14627871 1.12646434 1.10073823
 1.07125414 1.04030017 1.01000711 0.98161537 0.95531272 0.93062955
 0.90745242 0.88494628 0.86152039 0.83586184 0.80645175 0.77217203
 0.73272814 0.68866557 0.64165665 0.5935851  0.54557988 0.4971711
 0.44574408 0.38618944 0.31555739 0.23974024 0.1675932  

6
distance :  [66, 47, 68, 46, 69, 47] , avg :  57.0 <class 'numpy.float64'>
 True  0.7922010009869128
yhat :  [6.04806143e-01 6.27316143e-01 6.45684586e-01 6.60080619e-01
 6.70673390e-01 6.77632046e-01 6.81125735e-01 6.81323603e-01
 6.78394799e-01 6.72508470e-01 6.63833764e-01 6.52539827e-01
 6.38795808e-01 6.22770854e-01 6.04634112e-01 5.84554730e-01
 5.62701856e-01 5.39244636e-01 5.14352218e-01 4.88193750e-01
 4.60938379e-01 4.32755253e-01 4.03813519e-01 3.74282324e-01
 3.44330816e-01 3.14128143e-01 2.74324602e-01 2.33594996e-01
 1.92722950e-01 1.51725020e-01 1.12174360e-01 7.71225926e-02
 4.94830054e-02 2.95282398e-02 1.59255808e-02 7.20781846e-03
 2.29634611e-03 0.00000000e+00 6.26852216e-04 6.24225407e-03
 1.96442006e-02 4.19810338e-02 7.39214705e-02 1.14003897e-01
 1.59115093e-01 2.06634620e-01 2.54170401e-01 3.02724222e-01
 3.51271138e-01 3.99888926e-01 4.49042957e-01 4.98469024e-01
 5.47261796e-01 5.94326998e-01 6.39213259e-01 6.81656502e-01
 7.21507501e-01 7.57987788e-01 7.90

0


ZeroDivisionError: division by zero

In [19]:
#  for i in range(1, 11):
def main(rawData):
    lpf = LPF(rawData)
    yhat = normalization(savgol_filter(lpf.LPF, 51, 3))

    indices = find_peaks(yhat, threshold=0)[0]
    new_indices = np.array([])
    new_yhat = np.array([])
    print("indices : ", indices, type(indices))

    for i in indices:
        if (yhat[i] < 0.5):
            new_yhat = np.delete(yhat, i)
            new_indices = np.delete(indices, i)

#     print("new_indices : ", new_indices, type(new_indices))
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=new_yhat,
        mode='lines+markers',
        name='Original Plot'
    ))

    fig.add_trace(go.Scatter(
        x=indices,
        y=[new_yhat[j] for j in new_indices],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='cross'
        ),
        name='Detected Peaks'
    ))

    fig.show()

    fig.add_trace(go.Scatter(
        x=indices,
        y=[yhat[j] for j in indices],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='cross'
        ),
        name='Detected Peaks'
    ))

    fig.show()

In [24]:
def main(rawData) :
    
    lpf = LPF(rawData)
    yhat = normalization(savgol_filter(lpf.LPF, 51, 3))
    
    indices = find_peaks(yhat, threshold=0)[0]

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        y=yhat,
        mode='lines+markers',
        name='Original Plot'
    ))

    fig.add_trace(go.Scatter(
        x=indices,
        y=[yhat[j] for j in indices],
        mode='markers',
        marker=dict(
            size=8,
            color='red',
            symbol='cross'
        ),
        name='Detected Peaks'
    ))

    fig.show()

In [25]:
rawData_0 = pd.read_csv('0529Data/0529_user1/NormalGait1.csv')
rawData_1 = pd.read_csv('0529Data/0529_user1/AbnormalGait1.csv')
rawData_2 = pd.read_csv('0529Data/abnormal/abnormalWalk1.csv')

main(rawData_0)
main(rawData_1)
main(rawData_2)